Общая логика

In [1]:
from functools import reduce


class GrammarRule:
    left: str
    right: tuple[str]

    def __init__(self, l: str, r: list[str]) -> None:
        self.left, self.right = l, tuple(r)
        
    def __eq__(self, other) -> bool:
        return self.left == other.left and self.right == other.right
    
    def __hash__(self) -> int:
        return hash(self.left) ^ hash(self.right)


class Grammar:
    N: list[str]
    T: list[str]
    P: set[GrammarRule]
    S: str

    def __init__(
        self, N: list[str], T: list[str], P: set[GrammarRule], S: str
    ) -> None:
        self.N = N
        self.T = T
        self.P = P
        self.S = S

    def print(self, std=True, md=True):
        output = ""
        output += f"[{','.join(sorted(self.N))}]; [{','.join(sorted(self.T))}]; {self.S};\n"

        for N in sorted(self.N):
            P = list(filter(lambda x: x.left == N, self.P))
            if not len(P):
                continue

            output += "\n" + (
                N
                + " -> "
                + " | ".join(
                    sorted(
                        map(
                            lambda x: " ".join(x.right) if len(x.right) else "\epsilon",
                            P,
                        )
                    )
                )
            )
        if std:
            print(output)

        if md:
            output = "\n\n".join(
                map(
                    lambda x: (
                        "$" + x.replace("->", "\\rightarrow") + "$" if len(x) else ""
                    ),
                    output.split("\n"),
                )
            )

        return output

    def show(self):
        import IPython

        return IPython.display.Markdown(self.print(std=False))

    @staticmethod
    def from_text(text):
        N, T, a = text.split("\n")[0].split(" ")
        N, T = N[1:-1].split(","), T[1:-1].split(",")

        P = {
            GrammarRule(left, tuple([] if rule.startswith("\epsilon") else rule.split(" ")))
            for (left, right) in [
                s.split(" -> ")[:2]
                for s in filter(lambda x: len(x) > 0, text.split("\n")[1:])
            ]
            for rule in right.split(" | ")
        }

        return Grammar(N, T, P, a)

In [2]:
import IPython


def cmp_g(l_r, r_g):
    return l_r.print(False) == r_g.print(False)


def test(func, tests: tuple[Grammar], full: bool = False):
    output = "<table><thead><tr><td><div style='width:200px'>Вход</div></td><td><div style='width:200px'>Ожидаемый результат</div></td><td><div style='width:200px'>Действительный результат</div></td><td>Комментарий</td></tr></thead><tbody>"
    for test in tests:
        l = func(test[0])
        res = cmp_g(l, test[1])
        if not res or full:
            output += "<tr><td>\n\n{:}\n</td><td>\n\n{:}\n</td><td>\n\n{:}</td><td>\n\n{:}\n</td></tr>\n".format(
                test[0].print(std=False),
                test[1].print(std=False),
                l.print(std=False),
                "OK" if res else "ERROR",
            )
    output += "</tbody></table>"
    return output


In [3]:
grammar = Grammar(['E', 'T', 'F', 'G', 'R', 'U'], ['+', '-', '(', ')', 'a'], {
    GrammarRule('E', ['E', '+', 'T']),
    GrammarRule('E', ['T']),
    GrammarRule('T', ['T', '*', 'F']),
    GrammarRule('T', ['F']),
    GrammarRule('F', ['a']),
    GrammarRule('F', ['G']),
    GrammarRule('G', ['F']),
    GrammarRule('G', ['G', '+', 'R']),
    GrammarRule('G', []),
    GrammarRule('F', ['(', 'E', ')']),
    GrammarRule('U', ['F']),
}, 'E')

grammar.show()

$[E,F,G,R,T,U]; [(,),+,-,a]; E;$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

$U \rightarrow F$

Постройте программу, которая в качестве входа принимает приведенную КС-грамматику G и
преобразует ее в эквивалентную КС-грамматику G' без левой рекурсии.

In [4]:
def remove_left_recursion(g: Grammar) -> Grammar:
    P: list[GrammarRule] = list(g.P.copy())
    nts = g.N.copy()

    for i, n_i in enumerate(g.N):
        for n_j in g.N[:i]:
            rules = filter(
                lambda x: x.left == n_i and len(x.right) and x.right[0] == n_j, P
            )
            for rule in rules:
                P.remove(rule)
                m_rules = filter(lambda x: x.left == n_j, P)
                for m_rule in m_rules:
                    P.append(GrammarRule(n_i, m_rule.right + rule.right[1:]))
        rules = set(filter(lambda x: x.left == n_i, P))

        need_modify = any(len(rule.right) and rule.right[0] == n_i for rule in rules)
        if not need_modify:
            continue

        new_nt = n_i + "'"
        nts += [new_nt]

        for rule in rules:
            P.remove(rule)
            P += (
                [
                    GrammarRule(new_nt, rule.right[1:]),
                    GrammarRule(new_nt, rule.right[1:] + tuple([new_nt])),
                ]
                if len(rule.right) and rule.right[0] == n_i
                else [
                    GrammarRule(rule.left, rule.right),
                    GrammarRule(rule.left, rule.right + tuple([new_nt])),
                ]
            )

    return Grammar(nts, g.T.copy(), set(P), g.S)


display(grammar.show())
print("<---------------------------------------->")
remove_left_recursion(grammar).show()

$[E,F,G,R,T,U]; [(,),+,-,a]; E;$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

$U \rightarrow F$

<---------------------------------------->


$[E,E',F,G,G',R,T,T',U]; [(,),+,-,a]; E;$



$E \rightarrow T | T E'$

$E' \rightarrow + T | + T E'$

$F \rightarrow ( E ) | G | a$

$G \rightarrow ( E ) | ( E ) G' | G' | \epsilon | a | a G'$

$G' \rightarrow + R | + R G' | G' | \epsilon$

$T \rightarrow F | F T'$

$T' \rightarrow * F | * F T'$

$U \rightarrow ( E ) | ( E ) G' | G' | \epsilon | a | a G'$

In [5]:
report = test(remove_left_recursion, [
(
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
),
(
Grammar.from_text(
"""[A,S] [a,b,y] S

A -> S a
S -> S b | A y | b
"""),
Grammar.from_text(
"""[A,S,S'] [a,b,y] S

A -> S a
S -> b S' | b
S' -> b S' | a y S' | b | a y
"""),
),
(
Grammar.from_text(
"""[A] [+,-,a] A

A -> A + A | A - A | a
"""),
Grammar.from_text(
"""[A,A'] [+,-,a] A

A -> a | a A'
A' -> + A | + A A' | - A | - A A'
"""),
),
(
Grammar.from_text(
"""[A,B] [a,b] A

A -> B a | a
B -> A b | b
"""),
Grammar.from_text(
"""[A,B,B'] [a,b] A

A -> B a | a
B -> a b | a b B' | b | b B'
B' -> a b | a b B'
"""),
),
(
Grammar.from_text(
"""[A,B,C] [a,b] A

A -> B C | a
B -> C A | A b
C -> A B | C C | a
"""),
Grammar.from_text(
"""[A,B,B',C,C'] [a,b] A

A -> B C | a
B -> C A | a b | C A B' | a b B'
B' -> C b B' | C b
C -> a b C B | a b B' C B | a B | a | a b C B C' | a b B' C B C' | a B C' | a C'
C' -> A C B C' | A B' C B C' | C C' | A C B | A B' C B | C
"""),
)
], True)



IPython.display.Markdown(report)

<table><thead><tr><td><div style='width:200px'>Вход</div></td><td><div style='width:200px'>Ожидаемый результат</div></td><td><div style='width:200px'>Действительный результат</div></td><td>Комментарий</td></tr></thead><tbody><tr><td>

$[A]; [a]; A;$



$A \rightarrow a A$
</td><td>

$[A]; [a]; A;$



$A \rightarrow a A$
</td><td>

$[A]; [a]; A;$



$A \rightarrow a A$</td><td>

OK
</td></tr>
<tr><td>

$[A,S]; [a,b,y]; S;$



$A \rightarrow S a$

$S \rightarrow A y | S b | b$
</td><td>

$[A,S,S']; [a,b,y]; S;$



$A \rightarrow S a$

$S \rightarrow b | b S'$

$S' \rightarrow a y | a y S' | b | b S'$
</td><td>

$[A,S,S']; [a,b,y]; S;$



$A \rightarrow S a$

$S \rightarrow b | b S'$

$S' \rightarrow a y | a y S' | b | b S'$</td><td>

OK
</td></tr>
<tr><td>

$[A]; [+,-,a]; A;$



$A \rightarrow A + A | A - A | a$
</td><td>

$[A,A']; [+,-,a]; A;$



$A \rightarrow a | a A'$

$A' \rightarrow + A | + A A' | - A | - A A'$
</td><td>

$[A,A']; [+,-,a]; A;$



$A \rightarrow a | a A'$

$A' \rightarrow + A | + A A' | - A | - A A'$</td><td>

OK
</td></tr>
<tr><td>

$[A,B]; [a,b]; A;$



$A \rightarrow B a | a$

$B \rightarrow A b | b$
</td><td>

$[A,B,B']; [a,b]; A;$



$A \rightarrow B a | a$

$B \rightarrow a b | a b B' | b | b B'$

$B' \rightarrow a b | a b B'$
</td><td>

$[A,B,B']; [a,b]; A;$



$A \rightarrow B a | a$

$B \rightarrow a b | a b B' | b | b B'$

$B' \rightarrow a b | a b B'$</td><td>

OK
</td></tr>
<tr><td>

$[A,B,C]; [a,b]; A;$



$A \rightarrow B C | a$

$B \rightarrow A b | C A$

$C \rightarrow A B | C C | a$
</td><td>

$[A,B,B',C,C']; [a,b]; A;$



$A \rightarrow B C | a$

$B \rightarrow C A | C A B' | a b | a b B'$

$B' \rightarrow C b | C b B'$

$C \rightarrow a | a B | a B C' | a C' | a b B' C B | a b B' C B C' | a b C B | a b C B C'$

$C' \rightarrow A B' C B | A B' C B C' | A C B | A C B C' | C | C C'$
</td><td>

$[A,B,B',C,C']; [a,b]; A;$



$A \rightarrow B C | a$

$B \rightarrow C A | C A B' | a b | a b B'$

$B' \rightarrow C b | C b B'$

$C \rightarrow a | a B | a B C' | a C' | a b B' C B | a b B' C B C' | a b C B | a b C B C'$

$C' \rightarrow A B' C B | A B' C B C' | A C B | A C B C' | C | C C'$</td><td>

OK
</td></tr>
</tbody></table>

Вариант 5. Преобразование к приведенной грамматике.

Постройте программу, которая в качестве входа принимает произвольную КС-грамматику и преобразует ее в эквивалентную приведенную КС-грамматику.

Алгоритм состоит из трех этапов:
- Удалить из грамматики правила, содержащие непорождающие нетерминалы.
- Удалить из грамматики правила, содержащие недостижимые символы.
- Удалить из грамматики $\epsilon$-правила.

In [6]:
def remove_non_generative_nonterminals(g: Grammar) -> Grammar:
    N = set([g.S])

    modified = True
    while modified:
        added = set(
            map(
                lambda rule: rule.left,
                filter(
                    lambda p: p.right and all(x in g.T or x in N for x in p.right), g.P
                ),
            )
        )
        new_nt = N | added
        if len(new_nt) == len(N):
            modified = False

        N |= added

    P = set(
        rule
        for rule in g.P
        if rule.left in N and all(x in N or x in g.T for x in rule.right)
    )

    return Grammar(N, g.T, P, g.S)


display(grammar.show())
print("<---------------------------------------->")
remove_non_generative_nonterminals(grammar).show()

$[E,F,G,R,T,U]; [(,),+,-,a]; E;$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

$U \rightarrow F$

<---------------------------------------->


$[E,F,G,T,U]; [(,),+,-,a]; E;$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | \epsilon$

$T \rightarrow F$

$U \rightarrow F$

In [7]:
report = test(remove_non_generative_nonterminals, [
(
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
),
(
Grammar.from_text(
"""[A,B] [a,b] A
A -> B | A a | a
B -> B b
"""),
Grammar.from_text(
"""[A] [a,b] A
A -> A a | a
"""),
),
(
Grammar.from_text(
"""[A,B] [a,b] A
A -> B | A a | a
"""),
Grammar.from_text(
"""[A] [a,b] A
A -> A a | a
"""),
),
], True)


IPython.display.Markdown(report)

<table><thead><tr><td><div style='width:200px'>Вход</div></td><td><div style='width:200px'>Ожидаемый результат</div></td><td><div style='width:200px'>Действительный результат</div></td><td>Комментарий</td></tr></thead><tbody><tr><td>

$[A]; [a]; A;$



$A \rightarrow a A$
</td><td>

$[A]; [a]; A;$



$A \rightarrow a A$
</td><td>

$[A]; [a]; A;$



$A \rightarrow a A$</td><td>

OK
</td></tr>
<tr><td>

$[A,B]; [a,b]; A;$



$A \rightarrow A a | B | a$

$B \rightarrow B b$
</td><td>

$[A]; [a,b]; A;$



$A \rightarrow A a | a$
</td><td>

$[A]; [a,b]; A;$



$A \rightarrow A a | a$</td><td>

OK
</td></tr>
<tr><td>

$[A,B]; [a,b]; A;$



$A \rightarrow A a | B | a$
</td><td>

$[A]; [a,b]; A;$



$A \rightarrow A a | a$
</td><td>

$[A]; [a,b]; A;$



$A \rightarrow A a | a$</td><td>

OK
</td></tr>
</tbody></table>

In [8]:
def remove_unreachable_symbols(g: Grammar) -> Grammar:
    queue = [g.S]

    N, T, P = [g.S], [], set()

    while len(queue):
        A = queue.pop(0)

        for p in filter(lambda x: x.left == A, g.P):
            P.add(p)
            for symbol in p.right:
                if symbol in g.N and symbol not in N:
                    N.append(symbol)
                    queue.append(symbol)
                
                if symbol in g.T and symbol not in T:
                    T.append(symbol)

    return Grammar(N, T, P, g.S)

display(grammar.show())
print("<---------------------------------------->")
remove_unreachable_symbols(grammar).show()

$[E,F,G,R,T,U]; [(,),+,-,a]; E;$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

$U \rightarrow F$

<---------------------------------------->


$[E,F,G,R,T]; [(,),+,a]; E;$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

In [9]:
report = test(remove_unreachable_symbols, [
(
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
Grammar.from_text(
"""[A] [a] A

A -> a A
"""),
),
(
Grammar.from_text(
"""[A,B] [a,b] A
A -> A a | a
B -> A b
"""),
Grammar.from_text(
"""[A] [a] A
A -> A a | a
"""),
),
(
Grammar.from_text(
"""[A,B,C] [a,b,c] A
A -> A a | a
B -> C b 
C -> B c
"""),
Grammar.from_text(
"""[A] [a] A
A -> A a | a
"""),
),
(
Grammar.from_text(
"""[A,B,C] [a,b,c] A
A -> A a | a | B
B -> C b 
C -> B c
"""),
Grammar.from_text(
"""[A,B,C] [a,b,c] A
A -> A a | a | B
B -> C b 
C -> B c
"""),
),

], True)


IPython.display.Markdown(report)

<table><thead><tr><td><div style='width:200px'>Вход</div></td><td><div style='width:200px'>Ожидаемый результат</div></td><td><div style='width:200px'>Действительный результат</div></td><td>Комментарий</td></tr></thead><tbody><tr><td>

$[A]; [a]; A;$



$A \rightarrow a A$
</td><td>

$[A]; [a]; A;$



$A \rightarrow a A$
</td><td>

$[A]; [a]; A;$



$A \rightarrow a A$</td><td>

OK
</td></tr>
<tr><td>

$[A,B]; [a,b]; A;$



$A \rightarrow A a | a$

$B \rightarrow A b$
</td><td>

$[A]; [a]; A;$



$A \rightarrow A a | a$
</td><td>

$[A]; [a]; A;$



$A \rightarrow A a | a$</td><td>

OK
</td></tr>
<tr><td>

$[A,B,C]; [a,b,c]; A;$



$A \rightarrow A a | a$

$B \rightarrow C b $

$C \rightarrow B c$
</td><td>

$[A]; [a]; A;$



$A \rightarrow A a | a$
</td><td>

$[A]; [a]; A;$



$A \rightarrow A a | a$</td><td>

OK
</td></tr>
<tr><td>

$[A,B,C]; [a,b,c]; A;$



$A \rightarrow A a | B | a$

$B \rightarrow C b $

$C \rightarrow B c$
</td><td>

$[A,B,C]; [a,b,c]; A;$



$A \rightarrow A a | B | a$

$B \rightarrow C b $

$C \rightarrow B c$
</td><td>

$[A,B,C]; [a,b,c]; A;$



$A \rightarrow A a | B | a$

$B \rightarrow C b $

$C \rightarrow B c$</td><td>

OK
</td></tr>
</tbody></table>

In [10]:
def remove_epsilon_rules(g: Grammar) -> Grammar:
    N = set()

    modified = True
    while modified:
        added = set(
            map(
                lambda rule: rule.left,
                filter(lambda p: not p.right or N.issuperset(p.right), g.P),
            )
        )
        new_nt = N | added
        if len(new_nt) == len(N):
            modified = False

        N |= added

    P = g.P.copy()

    q = [(p, 0) for p in P]
    while len(q) > 0:
        p, idx = q.pop()

        new_idx, _ = next(
            filter(lambda t: t[1] in N, enumerate(p.right[idx:])), (-1, None)
        )

        if new_idx < 0:
            P.add(p)
            continue

        new_idx += idx

        q.append((p, new_idx + 1))
        q.append(
            (GrammarRule(p.left, p.right[:new_idx] + p.right[new_idx + 1 :]), new_idx)
        )

    old_n = len(P)
    P = set(filter(lambda x: x.right, P))

    S = g.S
    if len(P) != old_n:
        old_S = S
        S += "'"
        while any(p.left == S for p in P):
            S += "'"
        P |= {
            GrammarRule(S, tuple([old_S])),
            GrammarRule(S, tuple()),
        }
        N.add(S)

    return Grammar(N, g.T, list(P), S)


display(grammar.show())
print("<---------------------------------------->")
remove_epsilon_rules(grammar).show()

$[E,F,G,R,T,U]; [(,),+,-,a]; E;$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

$U \rightarrow F$

<---------------------------------------->


$[E,E',F,G,T,U]; [(,),+,-,a]; E';$



$E \rightarrow + | + T | E + | E + T | T$

$E' \rightarrow E | \epsilon$

$F \rightarrow ( ) | ( E ) | G | a$

$G \rightarrow + R | F | G + R$

$T \rightarrow * | * F | F | T * | T * F$

$U \rightarrow F$

In [11]:
report = test(remove_epsilon_rules, [
(
Grammar.from_text(
"""[S] [a,b] S

S -> a S b S | b S a S | \epsilon
"""),
Grammar.from_text(
"""[S',S] [a,b] S'

S' -> S | \epsilon
S -> a S b S | b S a S | a S b | a b S | a b | b S a | b a S | b a
"""),
),
(
Grammar.from_text(
"""[A,B,C,S] [a,b] S

S -> A B C
A -> B B | \epsilon
B -> C C | a
C -> A A | b
"""),
Grammar.from_text(
"""[A,B,C,S,S'] [a,b] S'

A -> B | B B
B -> C | C C | a
C -> A | A A | b
S -> A | A B | A B C | A C | B | B C | C
S' -> S | \epsilon
"""),
),
], True)


IPython.display.Markdown(report)

<table><thead><tr><td><div style='width:200px'>Вход</div></td><td><div style='width:200px'>Ожидаемый результат</div></td><td><div style='width:200px'>Действительный результат</div></td><td>Комментарий</td></tr></thead><tbody><tr><td>

$[S]; [a,b]; S;$



$S \rightarrow \epsilon | a S b S | b S a S$
</td><td>

$[S,S']; [a,b]; S';$



$S \rightarrow a S b | a S b S | a b | a b S | b S a | b S a S | b a | b a S$

$S' \rightarrow S | \epsilon$
</td><td>

$[S,S']; [a,b]; S';$



$S \rightarrow a S b | a S b S | a b | a b S | b S a | b S a S | b a | b a S$

$S' \rightarrow S | \epsilon$</td><td>

OK
</td></tr>
<tr><td>

$[A,B,C,S]; [a,b]; S;$



$A \rightarrow B B | \epsilon$

$B \rightarrow C C | a$

$C \rightarrow A A | b$

$S \rightarrow A B C$
</td><td>

$[A,B,C,S,S']; [a,b]; S';$



$A \rightarrow B | B B$

$B \rightarrow C | C C | a$

$C \rightarrow A | A A | b$

$S \rightarrow A | A B | A B C | A C | B | B C | C$

$S' \rightarrow S | \epsilon$
</td><td>

$[A,B,C,S,S']; [a,b]; S';$



$A \rightarrow B | B B$

$B \rightarrow C | C C | a$

$C \rightarrow A | A A | b$

$S \rightarrow A | A B | A B C | A C | B | B C | C$

$S' \rightarrow S | \epsilon$</td><td>

OK
</td></tr>
</tbody></table>

In [12]:
display(grammar.show())
print("<---------------------------------------->")
display(remove_left_recursion(grammar).show())
print("<---------------------------------------->")
display(remove_non_generative_nonterminals(remove_left_recursion(grammar)).show())
print("<---------------------------------------->")
display(remove_unreachable_symbols(remove_non_generative_nonterminals(remove_left_recursion(grammar))).show())
print("<---------------------------------------->")
display(remove_epsilon_rules(remove_unreachable_symbols(remove_non_generative_nonterminals(remove_left_recursion(grammar)))).show())

$[E,F,G,R,T,U]; [(,),+,-,a]; E;$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | G | a$

$G \rightarrow F | G + R | \epsilon$

$T \rightarrow F | T * F$

$U \rightarrow F$

<---------------------------------------->


$[E,E',F,G,G',R,T,T',U]; [(,),+,-,a]; E;$



$E \rightarrow T | T E'$

$E' \rightarrow + T | + T E'$

$F \rightarrow ( E ) | G | a$

$G \rightarrow ( E ) | ( E ) G' | G' | \epsilon | a | a G'$

$G' \rightarrow + R | + R G' | G' | \epsilon$

$T \rightarrow F | F T'$

$T' \rightarrow * F | * F T'$

$U \rightarrow ( E ) | ( E ) G' | G' | \epsilon | a | a G'$

<---------------------------------------->


$[E,E',F,G,T,U]; [(,),+,-,a]; E;$



$E \rightarrow T | T E'$

$E' \rightarrow + T | + T E'$

$F \rightarrow ( E ) | G | a$

$G \rightarrow ( E ) | \epsilon | a$

$T \rightarrow F$

$U \rightarrow ( E ) | \epsilon | a$

<---------------------------------------->


$[E,E',F,G,T]; [(,),+,a]; E;$



$E \rightarrow T | T E'$

$E' \rightarrow + T | + T E'$

$F \rightarrow ( E ) | G | a$

$G \rightarrow ( E ) | \epsilon | a$

$T \rightarrow F$

<---------------------------------------->


$[E,E'',F,G,T]; [(,),+,a]; E'';$



$E \rightarrow E' | T | T E'$

$E'' \rightarrow E | \epsilon$

$F \rightarrow ( ) | ( E ) | G | a$

$G \rightarrow ( ) | ( E ) | a$

$T \rightarrow F$